In [1]:
import os
import numpy as np
import pandas as pd
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Set the path to the dataset
data_dir = '/kaggle/input/animals-detection-images-dataset'

# Set the image size and batch size
image_size = (224, 224)
batch_size = 32

# Create an image data generator for the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Create a data generator for the training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Create an image data generator for the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Create a data generator for the validation data
val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Load the DenseNet121 model with pre-trained weights
base_model = DenseNet121(weights='imagenet', include_top=False)

# Add a global average pooling layer to the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a dense layer with the number of classes as the output size
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create a new model with the added layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model using the Adam optimizer and categorical crossentropy loss function
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up model checkpoint to save the best weights during training
checkpoint = ModelCheckpoint('animal_detection_model.h5', save_best_only=True)

# Train the model on the training data for 5 epochs, using the ModelCheckpoint callback
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[checkpoint])

Found 22566 images belonging to 80 classes.
Found 6505 images belonging to 80 classes.
29084464/29084464 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
706/706 [==============================] - 958s 1s/step - loss: 1.4365 - accuracy: 0.6396 - val_loss: 0.8591 - val_accuracy: 0.7500
Epoch 2/5
706/706 [==============================] - 791s 1s/step - loss: 0.7457 - accuracy: 0.7787 - val_loss: 0.7853 - val_accuracy: 0.7666
Epoch 3/5
706/706 [==============================] - 706s 1000ms/step - loss: 0.5828 - accuracy: 0.8121 - val_loss: 0.7488 - val_accuracy: 0.7742
Epoch 5/5
706/706 [==============================] - 690s 978ms/step - loss: 0.5455 - accuracy: 0.8200 - val_loss: 0.7735 - val_accuracy: 0.7783


In [2]:
import os
import numpy as np
import pandas as pd
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

data_dir = '/kaggle/input/animals-detection-images-dataset'

image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

base_model = DenseNet121(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('animal_detection_model.h5', save_best_only=True)

model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[checkpoint])

 * Serving Flask app '__main__'
 * Debug mode: off


In [1]:
from keras.models import load_model

# load the h5 model
model = load_model('./animal_detection_model.h5')

# Get a summary of the model's architecture
model.summary()

2023-04-16 15:38:34.628845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 15:38:34.858067: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 15:38:34.860234: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 15:38:38.340561: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1/conv (Conv2D)            (None, None, None,   9408        ['zero_padding2d[0][0]']         
                                64)                                                           

In [2]:
import h5py

# Open the h5 model file
with h5py.File('./animal_detection_model.h5', 'r') as f:
    # Print the groups in the file
    print("Groups in h5 file:", list(f.keys()))

    # Access a specific group
    group = f['group_name']

    # Print the datasets in the group
    print("Datasets in group:", list(group.keys()))

    # Access a specific dataset
    dataset = group['dataset_name']

    # Get the shape and datatype of the dataset
    print("Dataset shape:", dataset.shape)
    print("Dataset datatype:", dataset.dtype)

    # Get the values of the dataset
    print("Dataset values:", dataset[:])


Groups in h5 file: ['model_weights', 'optimizer_weights']


KeyError: "Unable to open object (object 'group_name' doesn't exist)"